In [1]:

from pathlib import Path
from zzb import build_injector, IngestService

inj = build_injector()
svc = inj.get(IngestService)


mdp = r"D:\codespace\fhfeishi\raga\scripts\PatentParser\full.md"
docs = svc.ingest_file(mdp, Path("mdp"))
print("ingested:", [d.doc_id for d in docs])

print("list:", svc.list_ingested()[:3])



23:05:30.626 [INFO    ] private_gpt.settings.settings_loader - Starting application with profiles=['default']


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\codespace\\fhfeishi\\data_db\\settings.yaml'

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embeds = HuggingFaceEmbedding(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    device="cpu",                 # 建议放顶层
    trust_remote_code=True,       # 建议放顶层
    cache_folder=r"E:\local_models\huggingface\cache",
    model_kwargs={"local_files_only": True},   # 允许联网 False
)

In [5]:
import os, torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM



import sys, transformers, huggingface_hub, sentence_transformers
print("py:", sys.executable)
print("tf:", transformers.__version__, "hub:", huggingface_hub.__version__, "st:", sentence_transformers.__version__)
print("HF_HOME:", os.getenv("HF_HOME"))
print("HF_ENDPOINT:", os.getenv("HF_ENDPOINT"))


# —— 离线&缓存环境（Windows 下也OK）——
os.environ["HF_HOME"] = r"E:\local_models\huggingface\cache"
# os.environ["HF_HUB_OFFLINE"] = "0"
# os.environ["TRANSFORMERS_OFFLINE"] = "1"
# os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"  # 关闭 symlink 警告
os.environ["HF_ENDPOINT"] = "https://mirrors.tuna.tsinghua.edu.cn/hugging-face/"


name_or_path = "Qwen/Qwen3-Embedding-0.6B"  # 也可换成本地目录（见方案2）
device = "cpu"  # 或 "cuda:0"（如果有GPU）

# 第一次若缓存不全，把 local_files_only 改为 False 联网补齐一次
tokenizer = AutoTokenizer.from_pretrained(
    name_or_path, trust_remote_code=True, local_files_only=False,
    cache_dir=os.environ["HF_HOME"],
)
model = AutoModelForCausalLM.from_pretrained(
    name_or_path, trust_remote_code=True, local_files_only=True,
    cache_dir=os.environ["HF_HOME"],
).to(device).eval()





py: d:\developer\miniconda\envs\langchain\python.exe
tf: 4.56.0 hub: 0.34.4 st: 5.0.0
HF_HOME: E:\local_models\huggingface\cache
HF_ENDPOINT: https://mirrors.tuna.tsinghua.edu.cn/hugging-face/


In [11]:
# test zzc
import os
from pathlib import Path
from zzc import build_injector, IngestService
from zzc.di import EmbeddingComponent  # 仅用于冒烟

# --- 环境变量 ---
os.environ['PGM_EMBED_MODEL'] = "Qwen/Qwen3-Embedding-0.6B"
os.environ['PGM_DEVICE'] = "cpu"             # 无GPU就改 "cpu"
os.environ['PGM_LOCAL_FILES_ONLY'] = "0"        # 首次跑若没缓存，改成 "0"
os.environ['PGM_CHROMA_DIR'] = "/data/pgm/chroma"
os.environ['PGM_CHROMA_COLLECTION'] = "pgm_collection"
os.environ['PGM_PERSIST_DIR'] = "/data/pgm/storage"
os.environ['PGM_SENT_WINDOW'] = "3"

inj = build_injector()

# 先冒烟：仅加载嵌入模型，验证“缓存+离线”是否OK
_ = inj.get(EmbeddingComponent)

svc: IngestService = inj.get(IngestService)

# 路径：确保与当前运行 OS 一致
mdp = r"D:\codespace\fhfeishi\raga\scripts\PatentParser\full.md"
p = Path(mdp)
assert p.exists(), f"文件不存在：{p}（注意当前运行环境的操作系统路径）"

docs = svc.ingest_file(p.name, p)
print([d.doc_id for d in docs])
print(svc.list_ingested()[:3])


TypeError: Transformer._load_model() got multiple values for argument 'cache_dir'

In [7]:
# test   zzc
import os 

# environment variables
os.environ['PGM_EMBED_MODEL'] = "Qwen/Qwen3-Embedding-0.6B"
os.environ['PGM_DEVICE'] = "cuda:0"
os.environ['PGM_LOCAL_FILES_ONLY'] = "1"    
os.environ['PGM_CHROMA_DIR'] = "/data/pgm/chroma"
os.environ['PGM_CHROMA_COLLECTION'] = "pgm_collection"
os.environ['PGM_PERSIST_DIR'] = "/data/pgm/storage"
os.environ['PGM_SENT_WINDOW'] = "3"


from pathlib import Path
from zzc import build_injector, IngestService

inj = build_injector()
svc: IngestService = inj.get(IngestService)

mdp = r"D:\codespace\fhfeishi\raga\scripts\PatentParser\full.md"
docs = svc.ingest_file(mdp, Path(mdp))
print([d.doc_id for d in docs])

print(svc.list_ingested()[:3])

# 删除
# svc.delete("demo.txt")


TypeError: Transformer._load_model() got multiple values for argument 'cache_dir'